In [ ]:
%display latex
%display default

## Précision numérique

Par défaut, les nombres flottants et les calculs flottants sont fait avec une précision de 53 bits

In [13]:
a= 3.0
a.parent()

Real Field with 53 bits of precision

In [17]:
a.precision()

53

In [9]:
(1.0/3.0).parent()

Real Field with 53 bits of precision

In [4]:
print(1/3, (1/3).parent())
print( (1/3).n(), (1/3).n().parent())
print(RR(1/3), RR(1/3).parent())


1/3 Rational Field
0.333333333333333 Real Field with 53 bits of precision
0.333333333333333 Real Field with 53 bits of precision


Les nombres flottants sont définis comme instance de l'objet `RealField`. On peut spécifier la précision de cette objet.

In [5]:
R=RealField()
R10=RealField(10)
R200=RealField(200)

R.precision(), R10.precision(), R200.precision()

(53, 10, 200)

In [ ]:
a   = R(3.513)
a10 = R10(3.513)
a200= R200(3.513)

a^100, a10^100, a200^100

Calcul de $0.3-0.2-0.1$ avec différentes précisions

In [ ]:
a,b,c=.3,.2,.1
a10,b10,c10=R10(.3),R10(.2),R10(.1)
a200,b200,c200=R200(.3),R200(.2),R200(.1)
a-b-c,a10-b10-c10,a200-b200-c200

**Attention:** 
- si on ne précise pas RealField, la précision est calculée automatiquement.
- si les calculs sont mixtes (opérandes avec différentes précisions), les calculs sont fait dans la plus petite précision

In [ ]:
a=0.5555555555554033333333323232323233232111117
a.precision(), a.parent()

In [ ]:
a10-b200-c200, a10-b10-c10

In [ ]:
a= 0.1e-20
b= 0.1
c= 0.3e-10
a+b+c-b,b-b+a+c,a+c+b-b

**A RETENIR**
> - `RealField(p)` est l'objet définissant les approximations flottantes à précision $p$
> - `numerical_approx(expr, p)` ou `n(expr,p)` ou `expr.n(p)` retourne l'approximation flottante de l'expression `expr` à précision `p`
> - par défaut `RealField(53)` est utilisé (déjà défini dans `RR`)

## Exemple de problème de précision

### calcul de racine carré

In [6]:
a=51
b=a^61
print("l'entier b est codée sur "+str( b.nbits()) + " bits")
r=sqrt(b)
r

l'entier b est codée sur 347 bits


1686961934066707040236155036109474174986501789839001*sqrt(51)

In [ ]:
s   = R(sqrt(b))
s200= R200(sqrt(b))
print(floor(s),floor(s200))
floor(s)^2 <= b <(floor(s)+1)^2,\
floor(s200)^2 <= b <=(floor(s200)+1)^2

## résolution de système d'équations linéaires 


In [ ]:
N=2
MS=MatrixSpace(QQ,N)
VS=VectorSpace(QQ,N)
A=MS.random_element(density=1,num_bound=10,den_bound=10)
b=VS.random_element(num_bound=100,den_bound=100)
V=var(list("x"+ str(i) for i in range(N)))
X=vector(V)
eq=list((A*X)[i]==b[i] for i in range(N))

### La fonction `solve`

elle prend une liste d'équations symboliques et une liste d'inconnues

In [ ]:
print("variables:",V)
print("Equations:",eq)
#for i in range(N):
#    print(eq[i])

In [ ]:
sol=solve(eq, V)
sol

Est-il possible de faire la même chose mais de faire le calcul numériquement ?

In [ ]:
eqf=list((A.n()*X)[i]==b[i].n() for i in range(N))
print("variables:",V)
print("Equations:",eqf)
#for i in range(N):
#    print(eqf[i])

In [ ]:
solf=solve(eqf,V)
solf

Quoi, le résultat est le même !!!

En fait, `solve` fait appel à une méthode de résolution symbolique. 
Cela implique que les réels sont transformées en nombre rationnel lors de la résolution. 

### l'algèbre linéaire

In [ ]:
show(eq)

In [ ]:
show(A, (V), b)

In [ ]:
y=A.solve_right(b)
show(y)

Pareil mais en numérique

In [ ]:
Af=A.numerical_approx()
bf=b.n()
show(Af,V,bf)

In [ ]:
yf=Af.solve_right(bf)
show(yf)

In [ ]:
# difference des solutions
y.n()-yf

Tout pareil mais en plus grand

In [ ]:
N=80
MS=MatrixSpace(QQ,N)
VS=VectorSpace(QQ,N)
A=MS.random_element(density=1,num_bound=100,den_bound=100)
b=VS.random_element(num_bound=100,den_bound=100)


In [ ]:
#print(A,b)
%time y=A.solve_right(b)

In [ ]:
y[0]

In [ ]:
print("num(y[0])=",y[0].numerator(),'\n')
print("den(y[0])=",y[0].denominator())

In [ ]:
Af=A.n()
bf=b.n()
%time yf= Af.solve_right(bf)

In [ ]:
print("yf[0]=",yf[0])
#erreur commise par la résolution numérique
y[0]-yf[0], (Af*yf)[0]- bf[0]

In [ ]:
N=20
A=matrix(N,N,lambda i,j : 1/(i+j+1))
b=vector([ZZ.random_element() for i in range(N)])
show(A)
show(b)

In [ ]:
Af=A.n()
bf=b.n()
%time y = A.solve_right(b)
%time yf= Af.solve_right(bf)

In [ ]:
### le résultat avec les flottants est complètement faux
print("yf[0]=",yf[0])
(Af*yf)[0], bf[0]

In [ ]:
### le résultat avec les rationnels est exact
print("y[0]=",y[0])
(A*y)[0],b[0],A*y==b

**A RETENIR**
- la fonction `solve` est une méthode de résolution symbolique
- les calculs avec les flottants peuvent être plus rapides mais aussi faux (surtout en algèbre linéaire)
- les calculs avec les rationnels entrainent un grossissement des données mais le résultat est toujours correct

## Limite du calcul symbolique

In [ ]:
var('a b c')
e1=a*c+b*c
e2=(a+b)*c
e1.show()
e2.show()
(e2 == e1.collect(c)).show()
(e1 == e2.expand()).show()
bool(e1==e2)

In [ ]:
e=(a-b)^2-a^2+2*a*b-b^2
e.show()
bool(e==0)

In [ ]:
var('x')
e= cos(x)^2+sin(x)^2
e, e.simplify_trig()

In [ ]:
e=sqrt(x^2)
bool(x==e), bool(x==e.simplify_full()),bool(x==e.canonicalize_radical())

### Explosion combinatoire des symboles

In [ ]:
var('x')
A=Matrix(2,2,[[cos(x),sin(x)],[-sin(x),cos(x)]])
show("A=",A,"  det(A)=",A.determinant())

In [ ]:
N=100
d=(A^N).determinant()
print("d=",d)

#A^N,(A^(N)).expand()

In [ ]:
%time print(bool(d==1))
%time print(bool(d.simplify_trig()==1))

### Pas toujours de solutions

In [ ]:
var('x')
(x^2+1).roots()

In [ ]:
E= (x+1)^5 - (x+2)^4
F= (x+1)^5 + (x+2)^4
EF=E*F
print(f"{E=}\n{F=}\n{EF=}")
EF.roots(x)

In [ ]:
E= (x+1)^10 - (x+2)^9
F= (x+1)^10 + (x+2)^9
EF=E*F
print(f"{E=}\n{F=}\n{EF=}")
EF.roots(x) ## aucunes solution trouvées alors que c'est un polynôme

Pour résoudre on passe du calcul symbolique au calcul algébrique.

Pour cela on se place dans la structure mathématique des polynomes qui sont des expressions symboliques ayant de bonne propriétés: en particulier une représentation canonique !!!

Un polynôme est une expression de la forme : $a_0+a_1X+\dots+a_nX^n$ avec $n$ une valeur connue et les coefficients $\{a_0,a_1,\dots,a_n\}$ qui sont tous du même type (Entier, Rationnel, Complexe, ...).

$\Rightarrow$ le vecteur $[a_0,a_1,\dots,a_n]$ est une représentation canonique de ce polynôme.

In [ ]:
# Ensemble des polynomes à coefficient dans QQ
# on choisit la variable X comme symbole des polynômes
PR.<X>=PolynomialRing(QQ) #ou PR.<X>=QQ[]

print("type de x: ",x.parent())
print("type de X: ",X.parent())
bool(X==x)

In [ ]:
print((2*X+4).parent()) 
print((2*x+4).parent()) 
print((2*X+ cos(3)).parent())
print((2*X+0.5).parent()) # attention coercion vers les polynomes à coefficient dans RR
RR

In [ ]:
# on convertit les expressions symboliques vers les polynomes en substituant 
# la variable x par celle des polynômes en X et en construisant le polynôme associé
polyE=PR(E.subs(x=X)) 
polyF=PR(F.subs(x=X))
polyEF=polyE*polyF
show(f"une expression symbolique: {E=}")
show(f"un polynomes: {polyE=}")


In [ ]:
# calcul des racines dans le domaine de définition du polynome (ici Q)
print(f"calcul des racines sur {polyEF.base_ring()}")
polyEF.roots()

In [ ]:
# calcul des racines dans C
R=polyEF.roots(CC)
R

In [ ]:
R[0][0],polyEF(R[0][0])

In [ ]:
polyEF(ComplexField(80)(R[0][0]))

In [ ]:
# calcul de la forme canonique d'un produit de polynômes
E1= (x+1)^100 - (x+2)^99
F1= (x+1)^100 + (x+2)^99
polyE1=PR(E.subs(x=X)) 
polyF1=PR(F.subs(x=X))

# via la representation par arbre (exp symbolique)
%time (E1*F1).expand() 
# via la representation par liste des coefficients
%time polyEF1=polyE1*polyF1 

**A RETENIR**
- le calcul symbolique permet de manipuler n'importe quelles expressions faisant intervenir des symboles et des fonctions 
- pas toujours possible de trouver des solutions avec uniquement des calculs symboliques
- l'utilisation de calcul algèbrique est souvent préférable si on connaît la structure mathématique sous-jacente (Ex. les polynômes, les matrices, ...)
- le calcul algébrique sera la plupart du temps plus rapide
- parfois on retrouve un mélange de calcul algébrique et numérique (résolution dans les complexes si le problème est défini sur les nombres rationnels)

## Les matrices

In [ ]:
A=matrix([[1,2],[3,4]])
B=matrix([[1,2,3],[4,5,6]])
print(A,B)
show(A,B)

In [ ]:
# addition de matrice
show(A+A)
# multiplication par un scalaire
show(3*A , A+A+A)

In [ ]:
show("A=",A)
show("B=",B)

A+B

### matrices et combinaisons linéaires

In [ ]:
# une combinaison linéaire de symboles
var('x y z')
e=[x,y,z]
3*e[0]+2*e[1]+5*e[2]

In [ ]:
u=matrix(1,3,[1,2,3])
v=matrix(3,1,e)
show(u,v)

In [ ]:
# le produit de matrice n*1 par 1*n est une combinaison linéaire
u*v

In [ ]:
# on peut augmenter le nombre de combinaisons linéaires (on ajoute une ligne à gauche)
B=matrix(2,3,[1,2,3,4,5,6])
show(B,v)
show(B*v)

In [ ]:
# on peut augmenter le nombre de combinaisons linéaires (on ajoute une colonne à droite)
var('x,y,z,a,b,c')
V=matrix(3,2,[[x,a],[y,b],[z,c]])
show(V)
show(B*V)

In [ ]:
# ATTENTION: les tailles des matrices doivent correspondre
show(V)
V*V

In [ ]:
# ATTENTION: le produit matriciel n'est pas commutatif
bool(V*B==B*V)

In [ ]:
show(V*B,B*V)

In [ ]:
# On peut transposer une matrice câd intervertir A[i,j] avec A[j,i]
show(V)
show(V.transpose())

In [ ]:
# le produit d'une matrice par sa transposée est toujours bien défini
show(V.transpose()*V)
show(V*V.transpose())

### Les espaces de matrices

In [ ]:
# le premier paramètres est le domaine des coefficients (optionnel)
# si il est présent il impose le type des coefficients
A=matrix([[1,2],[3,4]])
B=matrix(RR,[[1,2/3],[3,4]])
C=matrix([[1,2/3],[3,4]])
A.parent(),B.parent(),C.parent()

In [ ]:
matrix(ZZ,[[1,2/3],[3,4]])

In [ ]:
# le second/troisièmes paramètres sont les dimensions (optionnel)
# si ils sont présents ils imposent le nombre de coefficients (si aucun que des zéros)
A=matrix(ZZ,2,2)
B=matrix(ZZ,2,2,[[4,5],[6,7]])
C=matrix(ZZ,2,2,[4,5,6,7])
show(A)
show(B)
show(C)

In [ ]:
# Comme les matrices vivent dans un espace de matrice on peut définir cet objet
MS23=MatrixSpace(ZZ,2,3)
MS22=MatrixSpace(QQ,2,2)
show(MS22,"\tet\t",MS23)
MS22,MS23

In [ ]:
MS23.dimension() , MS23.dims()

In [ ]:
A=MS22.matrix([1,2,3,4])
A.parent()

In [ ]:
MS22.random_element()
show(_)

In [ ]:
MS10=MatrixSpace(ZZ,10,10)
MS10.random_element(density=1,x=0,y=100)
show(_)

In [ ]:
MS10=MS10.change_ring(QQ)
MS10.random_element(density=0.5,num_bound=100,den_bound=100)
show(_)

### un peu de calcul algébrique matriciel

In [ ]:
N=10
q=17
MS=MatrixSpace(GF(q),N,N)
A=MS.random_element()
show("A=",A)
show(f"det(A)={A.det()}")

In [ ]:
B=A.change_ring(ZZ)
show("B=",B)
show(f"det(B)={B.det()}")

In [ ]:
B.det()%q == A.det()

In [ ]:
show(A^-1)

In [ ]:
show(B^-1)

In [ ]:
(B^(-1)%q), A^-1
show(_)

In [ ]:
# Expliquer le résultat obtenu
(B^(-1)%q) ==  A^-1

In [ ]:
(B^(-1)%q).parent(),(A^-1).parent()

In [ ]:
(B^(-1)%q) ==  (A^-1).change_ring(B.base_ring())

In [ ]:
(B^(-1)).change_ring(A.base_ring()) ==  (A^-1)